# Time Series Machine Learning with XGBoost
- Dataset: Ice cream 🍦 sales
- Our **forecasting Horizon** will be 1 year (52 weeks) out into the future in which we will predict ice cream sales.
- Model(s): Linear Regression, XGBoost, or ARIMA

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt

# Model
import xgboost as xgb
import sklearn
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.metrics import mean_squared_error

## Import data

In [22]:
# Read in data
df = pd.read_excel(r'/home/jacquelinezhangg/workspace-jz/personal-projects/forecasting-ice-cream-sales/data/ice_cream_sales.xlsx')

In [23]:
df.head()

,Region,Periods,Item,$,Units
0,Albany/Schenectady/Troy,1 w/e 12/09/17,ICE CREAM,490.64,199.0
1,Albany/Schenectady/Troy,1 w/e 12/16/17,ICE CREAM,416.82,170.0
2,Albany/Schenectady/Troy,1 w/e 12/23/17,ICE CREAM,465.42,192.0
3,Albany/Schenectady/Troy,1 w/e 12/30/17,ICE CREAM,331.47,135.0
4,Albany/Schenectady/Troy,1 w/e 01/06/18,ICE CREAM,459.42,190.0


## Exploratory Data Analysis and Data Processing

Some data processing on the Date column

In [24]:
df['Periods'] = df['Periods'].str.split(' ', expand=True)[2]
df['Periods'] = pd.to_datetime(df['Periods'])

EDA
- Null values
- Duplicates
- Visualization of the trend and seasonality
- Possible outliers

In [44]:
df.isnull().sum() # No null values

Region     0
Periods    0
Item       0
$          0
Units      0
dtype: int64

In [45]:
df[df.duplicated(subset=['Region', 'Periods', 'Item'])] # No duplicates

,Region,Periods,Item,$,Units


In [43]:
def multiple_line_plot(df, title='Chart', date='date', width=800, height=400):
    """Transforms the dataframe by grouping by month and year and aggregating the numerical values. Then it plots that dataframe to see the monthly values.

    Args:
        df (pandas DataFrame): Data
        title (str, optional): Title of your plot. Defaults to 'Chart'.
        date (str, optional): Date column in your data. Defaults to 'date'.
        width (int, optional): Width of your plot. Defaults to 800.

    Returns:
        altair Chart: Plot of the monthly revenue
    """
    df[date] = pd.to_datetime(df[date])
    sum_of_revenue = df.groupby(by=df[date].dt.strftime('%Y %B')).sum(numeric_only=True).reset_index(drop=False)
    sum_of_revenue[date] = pd.to_datetime(sum_of_revenue[date]).sort_values().reset_index(drop=True)
    sum_of_revenue = sum_of_revenue.rename(columns={date: 'MONTH'})
    df_melt = sum_of_revenue.melt(id_vars='MONTH', var_name='VARIABLE', value_name='VALUE')

    return alt.Chart(df_melt).mark_line(
        color='lightblue',
        point=alt.OverlayMarkDef(color='darkblue'),
        width=30).encode(
            x=alt.X('MONTH:O', timeUnit='yearmonth', axis=alt.Axis(labelAngle=-90)),
            y='VALUE',
            color='VARIABLE'
            ).properties(title=title, width=width, height=height).interactive()

multiple_line_plot(df, title='Monthly Ice Cream Sales', date='Periods', width=800, height=400)

/home/jacquelinezhangg/anaconda3/envs/ml/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

Seems like there are seasonal fluctuations where Ice Cream sales are highest in the warmer times of the year and dip during season transitions

Outlier Detection

There are multiple stores, so we will aggregate all stores up to weekly-time-level data

In [27]:
# Transform to datetime dtype
df['Periods'] = pd.to_datetime(df['Periods'])

# Group by the dates being transformed in month and year
sum_of_revenue = df.groupby(by=df['Periods']).sum(numeric_only=True).reset_index(drop=False)

# Convert the date into datetime again to sort in order
sum_of_revenue['Periods'] = pd.to_datetime(sum_of_revenue['Periods']).sort_values().reset_index(drop=True)
sum_of_revenue = sum_of_revenue.rename(columns={'Periods': 'Date'})

In [28]:
# Create a copy to be able to manipulate the data
data_raw = sum_of_revenue.copy()
data_raw.head()

,Date,$,Units
0,2017-12-09,72767.839,31271.712
1,2017-12-16,61041.047,25734.947
2,2017-12-23,60838.874,25668.437
3,2017-12-30,51489.301,21898.136
4,2018-01-06,59647.489,25215.005


## Feature Engineering

#### Create features extracted from the time
- Month, Quarter, Year, Day of Year, and Seasons
- Data is at a weekly level

In [30]:
def create_features(data):
    """Creation of features (month, quarter, year, day of year, and season) using the date

    Args:
        data (pandas DataFrame): Time series data

    Returns:
        pandas DataFrame: Time series data with new features
    """
    data = data.copy()

    data['month'] = data['Date'].dt.month
    data['quarter'] = data['Date'].dt.quarter
    data['year'] = data['Date'].dt.year
    data['day_of_year'] = data['Date'].dt.day_of_year
    
    conditions = [
        (data['Date'].dt.month.isin(np.arange(3,6))),
        (data['Date'].dt.month.isin(np.arange(6,9))),
        (data['Date'].dt.month.isin(np.arange(9,12))),
        (data['Date'].dt.month.isin(np.append(np.arange(1,3), 12)))
    ]
    # 1 is Spring, 2 is Summer, 3 is Fall, and 4 is Winter
    choices = [1, 2, 3, 4]
    data['seasons'] = np.select(conditions, choices)
    return data

data = create_features(data_raw)

#### Lag
- Lag Features: the model will look back into the past and use it as a new feature to feed into the model
    - For example, if you like to forecast the sales in period `t`, you can use the sales of the previous month `t-1` as a feature

Visualizations
- Partial autocorrelation plot (correlogram): This will tell me the correlation of a lag along with all of the previous lags. Plotting the partial autocorrelation will help me choose which lag features to use.
    - Trying different lag lengths:
        - Day: looks ok but dips after 9 days
        - Month: does not look the best
        - Year: has good correlation throughout!
- Lag plot: Plot the unit and $ metrics against their lags to show the relationship. If it is a strong relationship, this shows their serial dependence.

In [47]:
def partial_autocorrelation_plot(data):
    """Plot autocorrelation out 3 lags (yearly) with a threshold of 0.3 (0.3 means they are loosely correlated)

    Args:
        data (pandas DataFrame): Data

    Returns:
        altair Chart: Autocorrelation chart of dollar sales and units 
    """
    temp = data.copy()
    for lag in np.arange(1, 10):
        # Multiply 4 to represent months
        # Multiply by 52 to represent a year
        temp[f'lag_Units_{lag}'] = temp['Units'].shift(periods=52*lag)
        temp[f'lag_$_{lag}'] = temp['$'].shift(periods=52*lag)

    # Calculate correlation
    temp = temp.corr(numeric_only=True).head(2)[['lag_Units_1', 'lag_$_1', 'lag_Units_2', 'lag_$_2',
        'lag_Units_3', 'lag_$_3', 'lag_Units_4', 'lag_$_4', 'lag_Units_5',
        'lag_$_5', 'lag_Units_6', 'lag_$_6', 'lag_Units_7', 'lag_$_7',
        'lag_Units_8', 'lag_$_8', 'lag_Units_9', 'lag_$_9']]

    # Split up by revenue and units
    temp_corr_rev = temp[temp.columns[temp.columns.str.contains('$', regex=False)]].drop('Units', axis=0)
    temp_corr_units = temp[temp.columns[temp.columns.str.contains('Units', regex=False)]].drop('Units', axis=0)
    
    # Altair plot
    rev_chart = alt.Chart(temp_corr_rev.melt()).mark_bar().encode(
        x='variable',
        y='value'
    )
    units_chart = alt.Chart(temp_corr_units.melt()).mark_bar().encode(
        x='variable',
        y='value'
    )
    line = alt.Chart(pd.DataFrame({'y': [0.3]})).mark_rule(color='red').encode(y='y')
    return alt.vconcat(rev_chart + line, units_chart + line)

partial_autocorrelation_plot(data)

/home/jacquelinezhangg/anaconda3/envs/ml/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In the figure above, the lags start dropping in correlation the farther we go out. Therefore we should pick the closest 3


In [33]:
def lag_plots(data, lag_num):
    """Plots a correlation chart between the units/$ metrics to their lags for how ever many lags specified

    Args:
        data (pandas DataFrame): Data
        lag_num (int): Number of lags. Level is yearly.

    Returns:
        altair Chart: Correlation lag plot
    """
    temp = data.copy()
    charts = []
    for lag in np.arange(1, lag_num+1):
        # Multiply by 52 to represent a year
        temp['lag_Units'] = temp['Units'].shift(periods=52*lag)
        temp['lag_$'] = temp['$'].shift(periods=52*lag)

        chart1 = alt.Chart(temp).mark_point().encode(x='lag_Units', y='Units')
        chart2 = alt.Chart(temp).mark_point(color = 'green').encode(x='lag_$', y='$')

        charts.append((
            chart1 + 
            chart1.transform_regression('lag_Units', 'Units').mark_line(color='light blue') + 
            chart2 + 
            chart2.transform_regression('lag_$', '$').mark_line(color='green')
            ).properties(title=f'Lag: {lag}'))

    # * means packaging arguments. In a function call, the * symbol can be used to pass a list of arguments as separate positional arguments
    # Also shared the x-axis among all charts
    return alt.vconcat(*charts).resolve_scale(
        x='shared'
    )

# Looking at correlation with 3 lag features
lag_plots(data, 3)

/home/jacquelinezhangg/anaconda3/envs/ml/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [34]:
def lag(data, lag_num):
    """Create yearly lag features

    Args:
        data (pandas DataFrame): Data
        lag_num (int): Number of lag features you want to create. Level is yearly.

    Returns:
        pandas DataFrame: Data with added lag features
    """
    data = data.copy()
    for lag in np.arange(1, lag_num+1):
        # Multiply by 52 to represent a year
        data[f'lag_Units_{lag}'] = data['Units'].shift(periods=52*lag)
        data[f'lag_$_{lag}'] = data['$'].shift(periods=52*lag)
    return data

# Create lag features and picking 3 lags
data = lag(data, 3)

Set the Date as the index for easier manipulation

In [35]:
# Set Date as the index
data = data.set_index('Date')
data.head()

,$,Units,month,quarter,year,day_of_year,seasons,lag_Units_1,lag_$_1,lag_Units_2,lag_$_2,lag_Units_3,lag_$_3
Date,,,,,,,,,,,,,
2017-12-09,72767.839,31271.712,12,4,2017,343,4,NaN,NaN,NaN,NaN,NaN,NaN
2017-12-16,61041.047,25734.947,12,4,2017,350,4,NaN,NaN,NaN,NaN,NaN,NaN
2017-12-23,60838.874,25668.437,12,4,2017,357,4,NaN,NaN,NaN,NaN,NaN,NaN
2017-12-30,51489.301,21898.136,12,4,2017,364,4,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-06,59647.489,25215.005,1,1,2018,6,4,NaN,NaN,NaN,NaN,NaN,NaN


## Visualize Feature and Target relationship
Looking to see where features have a strong relationship with the $ and unit metrics

In [37]:
alt.data_transformers.disable_max_rows()
alt.Chart(data).mark_boxplot(extent='min-max').encode(
    x='month',
    y='$'
).properties(title='Relationship of $ and Month')

alt.Chart(...)

In [40]:
alt.Chart(data).mark_boxplot(extent='min-max').encode(
    x='quarter',
    y='$'
).properties(title='Relationship of $ and Quarter')

alt.Chart(...)

In [42]:
alt.Chart(data).mark_boxplot(extent='min-max').encode(
    x='seasons',
    y='$'
).properties(title='Relationship of $ and Seasons')

alt.Chart(...)

## Split train and test data

In [299]:
# Finding the cutoff date for splitting up our dataset
testing = data.index.nunique()*0.2
training = data.index.nunique()*0.8
print('Cutoff point: ', data.index.max() - dt.timedelta(weeks=testing))

train = data[data.index < '2021-12-02'].copy()
test = data[data.index >= '2021-12-02'].copy()

Cutoff point:  2021-12-02 14:24:00


Split the train and test datasets into their own features and targets respectively

In [303]:
features = ['quarter', 'month', 'year', 'day_of_year', 'seasons', 'lag_Units_1', 'lag_$_1', 'lag_Units_2', 'lag_$_2', 'lag_Units_3', 'lag_$_3']
target = ['Units', '$']

X_train = train[features].copy()
y_train = train[target].copy()

X_test = test[features].copy()
y_test = test[target].copy()

## Model Creation, Training, and Validation
For XGBoost, we will use the regression model

In [304]:
# reg = xgb.XGBRegressor(n_estimators=1000, early_stopping_rounds=30, learning_rate=0.02)

# After grid search:
reg = xgb.XGBRegressor(learning_rate=0.05, max_depth=2, n_estimators=180)
reg.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=5)

[0]	validation_0-rmse:756079.23618	validation_1-rmse:842936.22939
[5]	validation_0-rmse:602476.15642	validation_1-rmse:685043.38442
[10]	validation_0-rmse:486525.18642	validation_1-rmse:553650.60600
[15]	validation_0-rmse:398853.53456	validation_1-rmse:451441.42187
[20]	validation_0-rmse:331703.84772	validation_1-rmse:373525.82188
[25]	validation_0-rmse:278847.63468	validation_1-rmse:315834.28253
[30]	validation_0-rmse:239269.05424	validation_1-rmse:272926.76544
[35]	validation_0-rmse:209363.39850	validation_1-rmse:238330.77120
[40]	validation_0-rmse:188145.36521	validation_1-rmse:210567.47337
[45]	validation_0-rmse:172728.81503	validation_1-rmse:193335.67633
[50]	validation_0-rmse:161597.09491	validation_1-rmse:180415.49304
[55]	validation_0-rmse:148928.30061	validation_1-rmse:166335.43709
[60]	validation_0-rmse:139398.07495	validation_1-rmse:156536.42985
[65]	validation_0-rmse:130913.51622	validation_1-rmse:150466.80536
[70]	validation_0-rmse:124876.65024	validation_1-rmse:145287.493

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=2, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=180, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

## Training Model

GridSearch

In [15]:
# Fine Tune the model
from sklearn.model_selection import GridSearchCV

xgb_reg = xgb.XGBRegressor()

# Already tried {'degree': [3, 4, 5, 6], 'C': [1, 2, 3, 4, 5, 7, 8, 9, 10, 11]}
param_grid = [
    {
        'max_depth': range(2, 10),
        'n_estimators': range(60, 220, 40),
        'learning_rate': [0.1, 0.01, 0.05],
    }
]

# The higher the verbose number, the more text output describing the process
grid_search = GridSearchCV(xgb_reg, param_grid, cv=5, refit=True, verbose=3, scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=(train=-4141969406.945, test=-19860593068.591) total time=   0.4s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=(train=-4820842360.670, test=-5778563153.657) total time=   0.0s
[CV 3/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=(train=-4088907184.801, test=-14946130302.316) total time=   0.0s
[CV 4/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=(train=-3860857543.655, test=-21197159099.574) total time=   0.0s
[CV 5/5] END learning_rate=0.1, max_depth=2, n_estimators=60;, score=(train=-4063908491.449, test=-8822278422.407) total time=   0.3s
[CV 1/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=(train=-2840070446.281, test=-19747002576.867) total time=   0.1s
[CV 2/5] END learning_rate=0.1, max_depth=2, n_estimators=100;, score=(train=-3036087927.619, test=-8286344747.534) total time=  

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid=[{'learning_rate': [0.1, 0.01, 0.05],
                          'max_depth': range(2, 10),
                          'n_estimators': range(60, 220, 40)}],
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=3)

It's a negative mean squared because of scikit-learn's implementation where they decided that higher values are better. These are loss functions so that means lower scores are better. So for it to be good, they flipped the sign for the mse value to be "higher"

In [19]:
# Best parameter after tuning
print(grid_search.best_params_)

# How our model looks with the best parameters
print(grid_search.best_estimator_)

# Best score after cross validation
neg_mse = grid_search.best_score_
rmse = np.sqrt(-neg_mse)
rmse

{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 180}
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=2, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=180, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)


118338.14654377222

### 1. Feature Importance and Root Mean Squared Error

Feature importances now that we have trained our model

In [330]:
feat_importance = pd.DataFrame(reg.feature_importances_, index=reg.feature_names_in_, columns=['importance'])

In [331]:
alt.Chart(feat_importance.reset_index().sort_values('importance')).mark_bar().encode(
    x='index',
    y='importance'
)

/home/jacquelinezhangg/anaconda3/envs/ml/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

Predict using the model

In [307]:
# Predict
reg.predict(X_test)

# Store these predictions
test[['prediction_units', 'prediction_$']] = reg.predict(X_test)

Compare the predictions with the actuals

In [308]:
# Merge predictions to the original data to be able to compare and plot it
data_merge = data.merge(test[['prediction_units', 'prediction_$']], how='left', left_index=True, right_index=True).reset_index()

# Plot prediction on line chart 
base = alt.Chart(data_merge[data_merge['Date'] > '2021-12-02']).encode(x=alt.X('Date:O', axis=alt.Axis(labelAngle=-90)))
alt.layer(
    base.mark_line(color='blue', strokeDash=[3,3]).encode(y='prediction_$'),
    base.mark_line(color='blue').encode(y='$'),
    base.mark_line(color='purple', strokeDash=[3,3]).encode(y='prediction_units'),
    base.mark_line(color='purple').encode(y='Units')
).interactive()


/home/jacquelinezhangg/anaconda3/envs/ml/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

Root Mean Squared Error

In [309]:
score_units = np.sqrt(mean_squared_error(test['Units'], test['prediction_units']))
score_rev = np.sqrt(mean_squared_error(test['$'], test['prediction_$']))
print(f'RMSE Score for units on test set: {score_units:0.2f}')
print(f'RMSE Score for revenue on test set: {score_rev:0.2f}')

# Old one before adding in lag and seasons was 
# RMSE Score for units on test set: 71601.73
# RMSE Score for revenue on test set: 230038.59

# Old one with lags shifte by a day
# RMSE Score for units on test set: 47981.77
# RMSE Score for revenue on test set: 146313.01

RMSE Score for units on test set: 68845.22
RMSE Score for revenue on test set: 209898.85


Error

In [310]:
error_rev = pd.DataFrame(abs(test['$'] - test['prediction_$']), columns=['Error']).sort_values(by='Error')
error_units = pd.DataFrame(abs(test['Units'] - test['prediction_units']), columns=['Error']).sort_values(by='Error')

In [311]:
# First 5 with least error
error_units.head()

,Error
Date,
2021-12-11,60.914469
2021-12-25,929.052875
2022-01-22,1957.108437
2021-12-18,2007.195469
2022-01-29,2495.389438


In [312]:
# First 5 with least error
error_rev.head()

,Error
Date,
2021-12-25,391.024875
2021-12-18,460.160250
2022-01-01,930.680562
2022-05-14,1930.921000
2021-12-04,4162.161750


### 2. Cross Fold Validation
- Regular cross fold validation will cause data leakage with time series data. This is because the testing window can be anywhere and the training can be anywhere as well. This can lead to several problems:
    1. The test data is taking place before the training data
    2. Data leakage: the training data can take place after the test data, which means the model is able to "peek" into the future
    3. The training data has gaps because the test data is taken from the middle of the time series

Therefore, we need a method where we can generate folds across a sliding window over time. The length of the training series will grow over time, with each subsequent fold retaining the full series history up to that point. The testing series length for each fold is constant.

---
Time Series Cross Validation
- This is better because it allows you to train your model on past data and evaluate on future data. It is different from other cross-validation techniques, which typically shuffle the data before splitting it into training and test sets, which can lead to **leakage of information** between the training and test sets
- Make sure to sort your DataFrame first or else it won't work
- The `TimeSeriesSplit` object is a generator so you have to loop over it and apply it to the training dataset
- It will return `train_idx` (indices of the dataframe for the training dataset) and `val_idx` (indices of the dataframe for the test dataset)


In [313]:
# The test_size parameter is how much you want to forecast out. We'll forecast out half a year (the data is in terms of weeks so we'll forecast 26 weeks out)
tss = TimeSeriesSplit(n_splits=5, test_size=26, gap=0)
data = data.sort_index()

Visualize how the `TimeSeriesSplit` cross validation works

In [314]:
# sourcery skip: convert-to-enumerate, remove-unused-enumerate
charts = []
fold = 0
for train_idx, val_idx in tss.split(data):
    train = data.iloc[train_idx]
    test = data.iloc[val_idx]
    chart1 = alt.Chart(train.reset_index()).mark_line().encode(x='Date', y='$')
    chart2 = alt.Chart(test.reset_index()).mark_line().encode(x='Date', y='$', color=alt.value("#FFAA00"))

    charts.append(chart1+chart2)
    fold +=1

# * means packaging arguments. In a function call, the * symbol can be used to pass a list of arguments as separate positional arguments
# Also shared the x-axis among all charts
alt.vconcat(*charts).resolve_scale(
    x='shared'
)

/home/jacquelinezhangg/anaconda3/envs/ml/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

Performing `TimeSeriesSplit` cross validation
- Overfitting means that the `RMSE` is similar in both the test and train datasets

In [315]:
model = xgb.XGBRegressor(learning_rate=0.05, max_depth=2, n_estimators=180)
rmse = []
plot=[]

for train_idx, val_idx in tss.split(data):
    train = data.iloc[train_idx].copy()
    test = data.iloc[val_idx].copy()

    X_train = train[features]
    y_train = train[target]

    X_test = test[features]
    y_test = test[target]

    # Fit model based on the training dataset
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=50)

    # Store predictions
    predictions = model.predict(X_test)

    # Root mean squared error
    rmse.append(np.sqrt(np.mean((predictions - y_test) ** 2, axis=0)))
    
    # Merge predictions to the original data to be able to compare and plot it
    test[['prediction_units', 'prediction_$']] = model.predict(X_test)
    data_merge = data.merge(test[['prediction_units', 'prediction_$']], how='left', left_index=True, right_index=True).reset_index()
    # Plot
    base = alt.Chart(data_merge[data_merge['Date'] > '2020-06-13']).encode(x=alt.X('Date')).properties(
    width=800,
    height=300)
    chart = alt.layer(
    base.mark_line(color='blue', strokeDash=[3,3]).encode(y='prediction_$'),
    base.mark_line(color='blue').encode(y='$'),
    base.mark_line(color='purple', strokeDash=[3,3]).encode(y='prediction_units'),
    base.mark_line(color='purple').encode(y='Units')
    )
    plot.append(chart)

alt.vconcat(*plot).resolve_scale(
    x='shared'
)

[0]	validation_0-rmse:716976.29909	validation_1-rmse:895068.68677
[50]	validation_0-rmse:164785.46006	validation_1-rmse:192581.51458
[100]	validation_0-rmse:96949.86054	validation_1-rmse:178287.08245
[150]	validation_0-rmse:77671.55180	validation_1-rmse:185318.19469
[179]	validation_0-rmse:69542.01072	validation_1-rmse:188458.14997
[0]	validation_0-rmse:748353.66011	validation_1-rmse:688227.87738
[50]	validation_0-rmse:168463.85481	validation_1-rmse:98425.33716
[100]	validation_0-rmse:102742.93551	validation_1-rmse:147912.67901
[150]	validation_0-rmse:83484.94080	validation_1-rmse:161231.71940
[179]	validation_0-rmse:76190.56941	validation_1-rmse:161794.90513
[0]	validation_0-rmse:739949.20958	validation_1-rmse:851592.52828
[50]	validation_0-rmse:163699.28835	validation_1-rmse:114580.46685
[100]	validation_0-rmse:103032.22399	validation_1-rmse:120648.28603
[150]	validation_0-rmse:86911.16346	validation_1-rmse:111583.70286
[179]	validation_0-rmse:80016.04623	validation_1-rmse:112347.070

/home/jacquelinezhangg/anaconda3/envs/ml/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [316]:
def display_scores(scores):
    print('Scores:', scores)
    print('Mean:', scores.mean())
    print('Standard deviation:', scores.std())

display_scores(np.array(rmse))

Scores: [[ 94190.83536086 249321.15968867]
 [ 83123.41248566 213179.93017763]
 [ 55285.7518218  148953.7326268 ]
 [ 51701.07291542  88726.83655924]
 [ 83596.10769941 264166.75334569]]
Mean: 133224.55926811794
Standard deviation: 76460.36268976625


## Predicting
1. Create a skeleton frame of dates we want to predict
2. Retrain on ALL data (including your test set) because we want to leverage all the data we have to forecast into the future

In [317]:
X_all = data[features]
y_all = data[target]

reg_final = xgb.XGBRegressor(learning_rate=0.05, max_depth=2, n_estimators=180)

reg_final.fit(X_all, y_all, eval_set=[(X_all, y_all)], verbose=50)

[0]	validation_0-rmse:774094.87428
[50]	validation_0-rmse:159606.73578
[100]	validation_0-rmse:110663.11168
[150]	validation_0-rmse:96930.02555
[179]	validation_0-rmse:90443.41977


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=2, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=180, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

Create the dataframe of future dates

In [324]:
print('Max date in dataframe: ', data.index.max())
future_df = pd.DataFrame(pd.date_range(start='2022-12-10', end='2023-12-09', freq='7D'))
future_df = future_df.rename(columns={0: 'Date'})

future_df['isFuture'] = True
data['isFuture'] = False

# Make sure the date isn't in the index yet to create the features
data = data.reset_index(drop=False)
data_and_future = pd.concat([data, future_df])

# Create features and lag features in `df_and_future` dataframe
data_and_future = create_features(data_and_future)
data_and_future = lag(data_and_future, 3)

Max date in dataframe:  260


In [325]:
# Set Date as the index
data_and_future = data_and_future.set_index('Date')
data_and_future.head()

,index,$,Units,month,quarter,year,day_of_year,seasons,lag_Units_1,lag_$_1,lag_Units_2,lag_$_2,lag_Units_3,lag_$_3,isFuture
Date,,,,,,,,,,,,,,,
2017-12-09,0.0,72767.839,31271.712,12,4,2017,343,4,NaN,NaN,NaN,NaN,NaN,NaN,False
2017-12-16,1.0,61041.047,25734.947,12,4,2017,350,4,NaN,NaN,NaN,NaN,NaN,NaN,False
2017-12-23,2.0,60838.874,25668.437,12,4,2017,357,4,NaN,NaN,NaN,NaN,NaN,NaN,False
2017-12-30,3.0,51489.301,21898.136,12,4,2017,364,4,NaN,NaN,NaN,NaN,NaN,NaN,False
2018-01-06,4.0,59647.489,25215.005,1,1,2018,6,4,NaN,NaN,NaN,NaN,NaN,NaN,False


In [326]:
future_w_feat = data_and_future.query('isFuture').copy()

In [327]:
future_w_feat.head()

,index,$,Units,month,quarter,year,day_of_year,seasons,lag_Units_1,lag_$_1,lag_Units_2,lag_$_2,lag_Units_3,lag_$_3,isFuture
Date,,,,,,,,,,,,,,,
2022-12-10,NaN,NaN,NaN,12,4,2022,344,4,47563.316,125240.979,44036.036,116237.727,34704.495,90403.069,True
2022-12-17,NaN,NaN,NaN,12,4,2022,351,4,45617.035,121098.254,40208.227,105332.259,32951.272,86408.317,True
2022-12-24,NaN,NaN,NaN,12,4,2022,358,4,42322.619,113390.397,34064.763,89222.358,28972.401,76374.202,True
2022-12-31,NaN,NaN,NaN,12,4,2022,365,4,39822.238,107773.954,38836.223,101500.678,35420.523,93865.903,True
2023-01-07,NaN,NaN,NaN,1,1,2023,7,4,45331.473,123902.778,43628.989,115033.692,32410.422,85832.661,True


In [328]:
# We only provide it with the features we trained on
future_w_feat[['pred_Units', 'pred_$']] = reg_final.predict(future_w_feat[features])

In [329]:
plotting_future = future_w_feat.reset_index()
plotting_future = plotting_future.melt(id_vars='Date', value_vars=['pred_Units', 'pred_$'])

plotting_past = data.melt(id_vars='Date', value_vars=['Units', '$'])

(
    alt.Chart(plotting_future).mark_line().encode(
    x='Date',
    y='value',
    color='variable'
)

+

alt.Chart(plotting_past).mark_line().encode(
    x='Date',
    y='value',
    color='variable'
)
)

/home/jacquelinezhangg/anaconda3/envs/ml/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

## Saving your Model

In [275]:
import pickle as pl

Improvement Ideas:
- Try 2-3 different models
- Drop rows where there's a null in the lag data? Or does XGBoost just ignore them?
- Moving average plot: To see what kind of trend a time series might have, we can use a moving average plot. To compute a moving average of a time series, we compute the average of the values within a sliding window of some defined width. Each point on the graph represents the average of all the values in the series that fall within the window on either side. The idea is to smooth out any short-term fluctuations in the series so that only long-term changes remain.